In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import KarateClub
from torch_geometric.data import DataLoader
from sklearn.model_selection import train_test_split

dataset = KarateClub()
data = dataset[0]

train_mask, test_mask = train_test_split(range(len(data.y)), test_size=0.2, random_state=42)
data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
data.test_mask = torch.tensor(test_mask, dtype=torch.bool)

class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

input_dim = dataset.num_features  
hidden_dim = 16  
output_dim = dataset.num_classes 
model = GNN(input_dim, hidden_dim, output_dim)


optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()  
model.train()
for epoch in range(100):  
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print(f'Test Accuracy: {acc:.4f}')


IndexError: The shape of the mask [27] at index 0 does not match the shape of the indexed tensor [34, 4] at index 0